# Специализация Data Science.
---

## Проект: Разведывательный анализ данных 
---

### **Общая информация**
На основе предоставленных данных (**stud_math.xls**) предстоит сформулировать гипотезы о связях различных переменных.

> _Легенда такова, что меня вдруг пригласили поучаствовать в одном из проектов UNICEF — международного подразделения ООН, чья миссия состоит в повышении уровня благополучия детей по всему миру._ 

#### <u>Задача моего проекта</u>:
Отследить влияние условий жизни учащихся на их успеваемость по математике.

#### Описание датасета:
- school — аббревиатура школы, в которой учится ученик
- sex — пол ученика ('F' - женский, 'M' - мужской)
- age — возраст ученика (от 15 до 22)
- address — тип адреса ученика ('U' - городской, 'R' - за городом)
- famsize — размер семьи('LE3' <= 3, 'GT3' >3)
- Pstatus — статус совместного жилья родителей ('T' - живут вместе 'A' - раздельно)
- Medu — образование матери (0 - нет, 1 - 4 класса, 2 - 5-9 классы, 3 - среднее специальное или 11 классов, 4 - высшее)
- Fedu — образование отца (0 - нет, 1 - 4 класса, 2 - 5-9 классы, 3 - среднее специальное или 11 классов, 4 - высшее)
- Mjob — работа матери ('teacher' - учитель, 'health' - сфера здравоохранения, 'services' - гос служба, 'at_home' - не работает, 'other' - другое)
- Fjob — работа отца ('teacher' - учитель, 'health' - сфера здравоохранения, 'services' - гос служба, 'at_home' - не работает, 'other' - другое)
- reason — причина выбора школы ('home' - близость к дому, 'reputation' - репутация школы, 'course' - образовательная программа, 'other' - другое)
- guardian — опекун ('mother' - мать, 'father' - отец, 'other' - другое)
- traveltime — время в пути до школы (1 - <15 мин., 2 - 15-30 мин., 3 - 30-60 мин., 4 - >60 мин.)
- studytime — время на учёбу помимо школы в неделю (1 - <2 часов, 2 - 2-5 часов, 3 - 5-10 часов, 4 - >10 часов)
- failures — количество внеучебных неудач (n, если 1<=n<=3, иначе 0)
- schoolsup — дополнительная образовательная поддержка (yes или no)
- famsup — семейная образовательная поддержка (yes или no)
- paid — дополнительные платные занятия по математике (yes или no)
- activities — дополнительные внеучебные занятия (yes или no)
- nursery — посещал детский сад (yes или no)
- higher — хочет получить высшее образование (yes или no)
- internet — наличие интернета дома (yes или no)
- romantic — в романтических отношениях (yes или no)
- famrel — семейные отношения (от 1 - очень плохо до 5 - очень хорошо)
- freetime — свободное время после школы (от 1 - очень мало до 5 - очень мого)
- goout — проведение времени с друзьями (от 1 - очень мало до 5 - очень много)
- health — текущее состояние здоровья (от 1 - очень плохо до 5 - очень хорошо)
- absences — количество пропущенных занятий
- score — баллы по госэкзамену по математике

### Выполним первичную обработку данных.
---

In [ ]:
%matplotlib inline

# импортируем все необходимые библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from scipy.stats import ttest_ind

In [ ]:
# читаем данные и составляем первое впечатление
df = pd.read_csv('stud_math.xls')
df.info()

#### Ошибки в данных
>_В столбцах **Fedu** и **famrel** присутствуют некорректные значения. Данные являются некорректными т.к. лежат за пределами возможных множеств значений переменных соответствующих столбцов._

>_Мною принято решение удалить ошибки путем заменой значений на NaN_

In [ ]:
# отец одного из учеников не может иметь образование в 10 раз выше высшего 
df.loc[df.Fedu == 40.0, 'Fedu'] = np.nan
# семейные отношения одного из учеников не может быть хуже чем очень плохо 
df.loc[df.famrel == -1.0, 'famrel'] = np.nan

#### Выбросы
>_Переменные **absences** (количество пропущенных занятий учеником) содержат выбросы._

In [ ]:
# посмотрим на распределение количества пропущенных занятий
print(df.absences.describe())
plt.hist(df.absences)
plt.show()

>_Мною принято решение удалить выбросы из данных о пропущенных занятиях учеников, путем заменой значений на NaN._

>_В данном случае выбросами считаю значение переменной которое превышает сумму среднего значения(mean) и трех среднеквадратичных отклонений(std)._

In [ ]:
# заменяем выбросы на np.nan
df.absences = df.absences.apply(
    lambda x: np.nan 
    if x > df.absences.mean() + 3*df.absences.std()
    else x
)
plt.hist(df.absences) # вновь глянем на распределение количества пропущенных занятий
plt.show()

#### Пропуски данных
>_т.к. датасет содержат пропуски, посмотрим на пропуски в датасете повнимательнее._

In [ ]:
omission = df.isna().sum(axis=1) # узнаем количество пропусков в каждой строке датасета
omission.describe()

In [ ]:
sns.boxplot(data=omission)
plt.show()

>_Мною принято решение исключить из датасета данные о учениках, где содержится слишком много пропусков._

In [ ]:
# удаляю строки eсли количество пропусков больше, чем значение верхнего квартиля  плюс 1.5 межквартильного размаха
perc25 = omission.quantile(0.25)
perc75 = omission.quantile(0.75)
IQR = perc75 - perc25

df = df.loc[omission.between(perc25 - 1.5*IQR, perc75 + 1.5*IQR)] # отфильтровывам данные

#### Загадочные данные
>_Отсутствует описание переменной '**studytime, granular**'. В данном столбце присутствуют не все значения (пропуски)._

In [ ]:
# посмотрим какие данные внутри
print(df['studytime, granular'].value_counts(dropna=False))

>_Данные состоят только из отрицательных значений кратных 3_

In [ ]:
# взглянем на статистические данные
print(df['studytime, granular'].describe())
plt.hist(df['studytime, granular'])
plt.show()

>_Вернусь к этой переменной, когда будет больше информации._

#### Замена переменных
>_Датасет содержит значения переменных типа 'Yes/No'. Для удобства дальнейшего анализа данных заменим 'Yes' на 1, 'No' на 0_

In [ ]:
def change_of_variables(col_name):
    df[col_name] = df[col_name].apply(
    lambda x: 1.0 
    if x == 'yes'
    else (0.0 if x == 'no' else x)
)

In [ ]:
# применим функцию change_of_variables для каждой переменной типа yes/no
col_list = ['schoolsup', 'famsup', 'paid', 'activities',  'nursery', 'higher', 'internet', 'romantic', ]
for col in col_list:
    change_of_variables(col)

### Проведем корреляционный анализ количественных переменных
---

In [ ]:
# вычисляем матрицу корреляций
correlation = df.corr()
# линейная корреляционная связь для всех количественных переменных
f, ax = plt.subplots(figsize =(9, 8))
sns.heatmap(correlation, ax = ax, cmap ="RdYlGn", linewidths = 0.1)
plt.show()

#### Предварительные выводы
- _Для **score** столбца слабо коррелирующие переменные: **age**, **Medu**, **Fedu**, **studytime**, **goout**, **schoolsup**, **higher**, **romantic**_

- _Переменные **Medu** и **Fedu** имеют высокий коэффициент корреляции (r=0.645)_

- _Для **score** столбца не коррелирующие переменные: **traveltime**, **famrel**, **freetime**, **health**, **absences**, **famsup**, **paid**, **activities**, **nursery**, **internet**_

- _Столбец '**studytime, granular**' полностью дублирует (r=-1) информацию столбца '**studytime**'._

- _Столбец **failures** (количество внеучебных неудач) имеет среднюю линейную обратную корелляционную связь (r=-0.332) с баллами учеников по госэкзамену по математике_

In [ ]:
sns.regplot(x=df.score, y=df.failures)
plt.xlabel('Баллы по госэкзамену по математике')
plt.ylabel('Количество внеучебных неудач')
plt.show()

### Проанализируем номинативные переменные
---
>_Номинативные переменные, которые осталось рассмотреть: **school**, **sex**, **address**, **famsize**, **Pstatus**, **Mjob**, **Fjob**, **reason**, **guardian**_

<span style="color:red">pss... Для построения графиков раскомментируй цикл for</span>

In [ ]:
def show_graph(col_name):
    sns.boxplot(x=df[col_name], y=df.score, data=df)
    plt.show()


# for col in ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian']:
#    show_graph(col)

>_Проверим, есть ли статистическая разница в распределении оценок по номинативным признакам, с помощью теста Стьюдента._

>*H_0: Pаспределения оценок по различным параметрам неразличимы*

In [ ]:
def get_stat_dif(column):
    cols = df.loc[:, column].value_counts().index[:10]
    combinations_all = list(combinations(cols, 2))
    for comb in combinations_all:
        if ttest_ind(df.loc[df.loc[:, column] == comb[0], 'score'], 
                        df.loc[df.loc[:, column] == comb[1], 'score']).pvalue \
            <= 0.05/len(combinations_all): # Учли поправку Бонферони
            print('Найдены статистически значимые различия для колонки', column)
            break
            
for col in ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian']:
    get_stat_dif(col)

In [ ]:
show_graph('Mjob')

In [ ]:
df_for_model = df.loc[:, ['age', 'Medu', 'Mjob', 'studytime', 'failures', 'schoolsup', 'higher', 'romantic', 'goout']]
df_for_model.head()

#### Предварительные выводы
- _Серьезно отличается всего один параметр: **Mjob**. Оставим эту переменную в датасете для дальнейшего построения модели_

## Выводы
---
- В данных присутствуют пустые значения
- В столбцах **Fedu** и **famrel** присутствуют некорректные значения. Выполнена замена на NaN
- Выбросы найдены (и заменены на NaN) лишь в данных переменной **absences** (количество пропущенных занятий учеником)
- Данные учеников, которые содержат слишком много пропусков, исключены из датасета
- Столбец **studytime, granular** полностью дублирует (r=-1) информацию столбца **studytime** (время на учёбу помимо школы в неделю). Столбец **studytime, granular** можно изключить из дальнейшего рассмотрения
- Переменные **Medu** (образование матери) и **Fedu** (образование отца) имеют высокий коэффициент корреляции (r=0.645), что может говорить о не случайном выборе супругов. Принадлежность к одной социальной группе
- Столбец **failures** (количество внеучебных неудач) имеет среднюю линейную обратную корелляционную связь (r=-0.332) с баллами учеников по госэкзамену по математике (**score**). Это может говорить о взаимосвязи опыта неудач с получаемыми результатами. 
- Самые важные параметры, которые предлагается использовать в дальнейшем для построения модели:
    + **age** - возраст ученика
    + **Medu** - образование матери
    + **Mjob** - работа матери
    + **studytime** - время на учёбу помимо школы в неделю
    + **failures** - количество внеучебных неудач
    + **schoolsup** - дополнительная образовательная поддержка
    + **higher** - хочет получить высшее образование
    + **romantic** - в романтических отношениях
    + **goout** - проведение времени с друзьями